In [1]:
from __future__ import annotations

from pathlib import Path

import dill
import numpy as np
import rich
from matbench_discovery.enums import MbdKey
from matbench_discovery.metrics import stable_metrics
from pymatviz import Key
from tqdm.auto import tqdm

results_dir = Path(
    "/net/csefiles/coc-fung-cluster/nima/shared/repositories/jmp-backbone/notebooks/results/epoch0-step57444/"
)
results = list(results_dir.glob("*.dill"))
print(f"Found {len(results)} results")

predicted_eform_list: list[float] = []
true_eform_list: list[float] = []

predicted_ehull_list: list[float] = []
true_ehull_list: list[float] = []

for result_path in tqdm(results):
    with result_path.open("rb") as f:
        result = dill.load(f)

    # Ignore non-unique prototypes
    if not result["metadata"][Key.uniq_proto.value]:
        continue

    eform_pred = result["energy"]
    predicted_eform_list.append(eform_pred)

    eform_true = result["metadata"][MbdKey.e_form_wbm.value]
    true_eform_list.append(eform_true)

    hull_true = result["metadata"][MbdKey.each_true.value]
    true_ehull_list.append(hull_true)

    hull_pred = hull_true + (eform_pred - eform_true)
    predicted_ehull_list.append(hull_pred)

predicted_eform = np.array(predicted_eform_list)
true_eform = np.array(true_eform_list)

predicted_ehull = np.array(predicted_ehull_list)
true_ehull = np.array(true_ehull_list)

rich.print(stable_metrics(true_ehull, predicted_ehull))

Found 900 results


  0%|          | 0/900 [00:00<?, ?it/s]

{
    'F1': 0.5821596244131455,
    'DAF': 4.266259432267337,
    'Precision': 0.6739130434782609,
    'Recall': 0.512396694214876,
    'Accuracy': 0.8838120104438643,
    'TPR': 0.512396694214876,
    'FPR': 0.046511627906976744,
    'TNR': 0.9534883720930233,
    'FNR': 0.48760330578512395,
    'TP': 62,
    'FP': 30,
    'TN': 615,
    'FN': 59,
    'MAE': 0.17895551253438688,
    'RMSE': 0.2930696514331469,
    'R2': -1.4161107520383238
}